In [ ]:
# Install old version of scikit-learn, see https://github.com/SeldonIO/seldon-core/issues/2059
!pip install -UIv scikit-learn==0.20.3

In [ ]:
!pip install azure-storage-file-datalake azure-identity azure-storage-blob pandas joblib

In [ ]:
### ENTER YOUR DETAILS  ###

storage_account_name = ""
client_id = ""
tenant_id = ""
client_secret = ""                                      # client secret value of the service principal
connection_string = ""                                  # blob storage connection string

# run `oc whoami --show-token` to get your token
# do not use quotes
%env OPENSHIFT_TOKEN=
%env STORAGE_ACCOUNT_NAME=

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

def initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id):
    
    try:  
        global service_client

        credential = ClientSecretCredential(tenant_id, client_id, client_secret)

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=credential)
    
    except Exception as e:
        print(e)
        
def download_file_from_directory(dataset):
    try:
        file_system_client = service_client.get_file_system_client(file_system="mycontainer")
        directory_client = file_system_client.get_directory_client("sample")
        
        local_file = open(dataset,'wb')

        file_client = directory_client.get_file_client(dataset)

        download = file_client.download_file()

        downloaded_bytes = download.readall()

        local_file.write(downloaded_bytes)

        local_file.close()

    except Exception as e:
     print(e)

In [ ]:
# Initialize and download Iris dataset from Azure Data Lake
initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id)
download_file_from_directory("iris.data")

# Read training data set
train_df = pd.read_csv("iris.data", header=None, names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
y = pd.factorize(train_df["class"])[0]
train_df.pop("class")
X = train_df.values

# Train model
clf = LogisticRegression()
clf.fit(X,y)

# Test model
print(X[0:2])
print(clf.predict(X[0:2]))

# Save model to local disk
dump(clf, 'model.joblib')

In [ ]:
# Save model to Azure Blob Storage
local_file_name = "model.joblib"
upload_path = "sklearn/model.joblib"

try:
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container="mycontainer", blob=upload_path)

    print("\nUploading to Azure Storage as blob:\n\t" + upload_path)

    # Upload the created file
    with open(local_file_name, "rb") as data:
        blob_client.upload_blob(data)
    
except Exception as ex:
    print('Exception:')
    print(ex)

In [ ]:
%%bash

curl -O https://mirror.openshift.com/pub/openshift-v4/clients/oc/4.6/linux/oc.tar.gz
tar xzf oc.tar.gz
cp oc /opt/app-root/bin/

In [ ]:
%%bash

# Test oc
oc login --server https://openshift.default.svc.cluster.local --insecure-skip-tls-verify --token=$OPENSHIFT_TOKEN

# Run model in Seldon
oc apply -n odh -f - <<EOF
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: example
spec:
  name: iris
  predictors:
    - graph:
        children: []
        implementation: SKLEARN_SERVER
        modelUri: https://$STORAGE_ACCOUNT_NAME.blob.core.windows.net/mycontainer/sklearn/model.joblib
        name: classifier
      name: default
      replicas: 1
EOF

In [ ]:
%%bash

# Test model in Seldon
MODEL_URL=example-default.odh.svc.cluster.local:8000
curl -X POST $MODEL_URL/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{ "data": { "ndarray": [[1,2,3,4]] } }'